In [ ]:
"""
pip install transformers==4.44.2
pip install accelerate==0.33.0
pip install peft==0.14.0
pip install datasets
pip install sentencepiece
pip install bitsandbytes
pip install hf-xet==1.2.0
"""

"""
(.venv) PS D:\claimpkg\claimpkg-clone> huggingface-cli login
⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth loogin' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.
Enter your token (input will not be visible):
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `Llama-3.2-1B-TNG-token` has been saved to C:\Users\tungq\.cache\huggingface\stored_tokens
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\tungq\.cache\huggingface\token
Login successful.
The current active token is: `Llama-3.2-1B-TNG-token`
"""

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_ID = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
)

In [ ]:
# Check if model is runable now

text = "Claim: Barack Obama was born in Hawaii."
inputs = tokenizer(text, return_tensors="pt").to(model.device)

out = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(out[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Claim: Barack Obama was born in Hawaii. Fact: Barack Obama was born in Honolulu, Hawaii on August 4, 1961. He


In [4]:
from peft import LoraConfig, get_peft_model

lora = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5485
